---


title: "ADPC"


order: 2


---


The Non-compartmental analysis (NCA) ADaM uses the CDISC Implementation Guide (<https://www.cdisc.org/standards/foundational/adam/adamig-non-compartmental-analysis-input-data-v1-0>). This example presented uses underlying `EX` and `PC` domains where the `EX` and `PC` domains represent data as collected and the `ADPC` ADaM is output. For more details see the `{admiral}` [vignette](https://pharmaverse.github.io/admiral/articles/pk_adnca.html){target="_blank"}.


## First Load Packages


First we will load the packages required for our project. We will use `{admiral}` for the creation of analysis data. `{admiral}` requires `{dplyr}`, `{lubridate}` and `{stringr}`. We will use `{metacore}` and `{metatools}` to store and manipulate metadata from our specifications. We will use `{xportr}` to perform checks on the final data and export to a transport file.


The source SDTM data will come from the CDISC pilot study data stored in `{pharmaversesdtm}`.


In [ ]:
#| label: Load Packages
# Load Packages
library(admiral)
library(dplyr)
library(lubridate)
library(stringr)
library(metacore)
library(metatools)
library(xportr)
library(pharmaversesdtm)
library(pharmaverseadam)


## Next Load Specifications for Metacore


We have saved our specifications in an Excel file and will load them into `{metacore}` with the `metacore::spec_to_metacore()` function.


In [ ]:
#| label: Load Specs
#| warning: false
# ---- Load Specs for Metacore ----

metacore <- spec_to_metacore("../metadata/pk_spec.xlsx") %>%
  select_dataset("ADPC")

## Load Source Datasets


We will load are SDTM data from `{pharmaversesdtm}`. The main components of this will be exposure data from `EX` and pharmacokinetic concentration data from `PC`. We will use `ADSL` for baseline characteristics and we will derive additional baselines from vital signs `VS`.


In [5]:
#| label: Load Source
# ---- Load source datasets ----
# Load PC, EX, VS, LB and ADSL
data("pc")
data("ex")
data("vs")

data("adsl")

ex <- convert_blanks_to_na(ex)
pc <- convert_blanks_to_na(pc)
vs <- convert_blanks_to_na(vs)


In [21]:
head(ex,3)

STUDYID,DOMAIN,USUBJID,EXSEQ,EXTRT,EXDOSE,EXDOSU,EXDOSFRM,EXDOSFRQ,EXROUTE,VISITNUM,VISIT,VISITDY,EXSTDTC,EXENDTC,EXSTDY,EXENDY
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
CDISCPILOT01,EX,01-701-1015,1,PLACEBO,0,mg,PATCH,QD,TRANSDERMAL,3,BASELINE,1,2014-01-02,2014-01-16,1,15
CDISCPILOT01,EX,01-701-1015,2,PLACEBO,0,mg,PATCH,QD,TRANSDERMAL,4,WEEK 2,14,2014-01-17,2014-06-18,16,168
CDISCPILOT01,EX,01-701-1015,3,PLACEBO,0,mg,PATCH,QD,TRANSDERMAL,12,WEEK 24,168,2014-06-19,2014-07-02,169,182


In [13]:
colnames(ex)

[1] "STUDYID"  "DOMAIN"   "USUBJID"  "EXSEQ"    "EXTRT"    "EXDOSE"  
 [7] "EXDOSU"   "EXDOSFRM" "EXDOSFRQ" "EXROUTE"  "VISITNUM" "VISIT"   
[13] "VISITDY"  "EXSTDTC"  "EXENDTC"  "EXSTDY"   "EXENDY"

In [20]:
head(pc, 3)

STUDYID,DOMAIN,USUBJID,PCSEQ,PCTESTCD,PCTEST,PCORRES,PCORRESU,PCSTRESC,PCSTRESN,PCSTRESU,PCNAM,PCSPEC,PCLLOQ,VISIT,VISITNUM,PCDTC,PCDY,PCTPT,PCTPTNUM
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
CDISCPILOT01,PC,01-701-1015,1,XAN,XANOMELINE,<BLQ,ug/ml,<BLQ,0,ug/ml,Imaginary Labs,PLASMA,0.01,BASELINE,3,2014-01-01T23:30:00,-1,Pre-dose,-0.50
CDISCPILOT01,PC,01-701-1015,2,XAN,XANOMELINE,<BLQ,ug/ml,<BLQ,NA,ug/ml,Imaginary Labs,PLASMA,0.01,BASELINE,3,2014-01-02T00:05:00,1,5 Min Post-dose,0.08
CDISCPILOT01,PC,01-701-1015,3,XAN,XANOMELINE,<BLQ,ug/ml,<BLQ,NA,ug/ml,Imaginary Labs,PLASMA,0.01,BASELINE,3,2014-01-02T00:30:00,1,30 Min Post-dose,0.50


In [14]:
colnames(pc)

[1] "STUDYID"  "DOMAIN"   "USUBJID"  "PCSEQ"    "PCTESTCD" "PCTEST"  
 [7] "PCORRES"  "PCORRESU" "PCSTRESC" "PCSTRESN" "PCSTRESU" "PCNAM"   
[13] "PCSPEC"   "PCLLOQ"   "VISIT"    "VISITNUM" "PCDTC"    "PCDY"    
[19] "PCTPT"    "PCTPTNUM"

In [19]:
head(vs, 3)

STUDYID,DOMAIN,USUBJID,VSSEQ,VSTESTCD,VSTEST,VSPOS,VSORRES,VSORRESU,VSSTRESC,...,VSBLFL,VISITNUM,VISIT,VISITDY,VSDTC,VSDY,VSTPT,VSTPTNUM,VSELTM,VSTPTREF
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
CDISCPILOT01,VS,01-701-1015,1,DIABP,Diastolic Blood Pressure,SUPINE,64,mmHg,64,...,NA,1,SCREENING 1,-7,2013-12-26,-7,AFTER LYING DOWN FOR 5 MINUTES,815,PT5M,PATIENT SUPINE
CDISCPILOT01,VS,01-701-1015,2,DIABP,Diastolic Blood Pressure,STANDING,83,mmHg,83,...,NA,1,SCREENING 1,-7,2013-12-26,-7,AFTER STANDING FOR 1 MINUTE,816,PT1M,PATIENT STANDING
CDISCPILOT01,VS,01-701-1015,3,DIABP,Diastolic Blood Pressure,STANDING,57,mmHg,57,...,NA,1,SCREENING 1,-7,2013-12-26,-7,AFTER STANDING FOR 3 MINUTES,817,PT3M,PATIENT STANDING


In [16]:
colnames(vs)

[1] "STUDYID"  "DOMAIN"   "USUBJID"  "VSSEQ"    "VSTESTCD" "VSTEST"  
 [7] "VSPOS"    "VSORRES"  "VSORRESU" "VSSTRESC" "VSSTRESN" "VSSTRESU"
[13] "VSSTAT"   "VSLOC"    "VSBLFL"   "VISITNUM" "VISIT"    "VISITDY" 
[19] "VSDTC"    "VSDY"     "VSTPT"    "VSTPTNUM" "VSELTM"   "VSTPTREF"

## Derivations


### Derive PC Dates


Here we use `{admiral}` functions for working with dates and we will also create a nominal time from first dose `NFRLT` for `PC` data based on `PCTPTNUM`.


In [6]:
#| label: PC Dates

# Get list of ADSL vars required for derivations
adsl_vars <- exprs(TRTSDT, TRTSDTM, TRT01P, TRT01A)

pc_dates <- pc %>%
  # Join ADSL with PC (need TRTSDT for ADY derivation)
  derive_vars_merged(
    dataset_add = adsl,
    new_vars = adsl_vars,
    by_vars = exprs(STUDYID, USUBJID)
  ) %>%
  # Derive analysis date/time
  # Impute missing time to 00:00:00
  derive_vars_dtm(
    new_vars_prefix = "A",
    dtc = PCDTC,
    time_imputation = "00:00:00"
  ) %>%
  # Derive dates and times from date/times
  derive_vars_dtm_to_dt(exprs(ADTM)) %>%
  derive_vars_dtm_to_tm(exprs(ADTM)) %>%
  derive_vars_dy(reference_date = TRTSDT, source_vars = exprs(ADT)) %>%
  # Derive event ID and nominal relative time from first dose (NFRLT)
  mutate(
    EVID = 0,
    DRUG = PCTEST,
    NFRLT = if_else(PCTPTNUM < 0, 0, PCTPTNUM), .after = USUBJID
  )


In [18]:
head(pc_dates, 3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,PCTPTNUM,TRTSDT,TRTSDTM,TRT01P,TRT01A,ADTM,ATMF,ADT,ATM,ADY
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<dbl>,<date>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<date>,<time>,<dbl>
CDISCPILOT01,PC,01-701-1015,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,-0.50,2014-01-02,2014-01-02,Placebo,Placebo,2014-01-01 23:30:00,NA,2014-01-01,23:30:00,-1
CDISCPILOT01,PC,01-701-1015,0,XANOMELINE,0.08,2,XAN,XANOMELINE,<BLQ,...,0.08,2014-01-02,2014-01-02,Placebo,Placebo,2014-01-02 00:05:00,NA,2014-01-02,00:05:00,1
CDISCPILOT01,PC,01-701-1015,0,XANOMELINE,0.50,3,XAN,XANOMELINE,<BLQ,...,0.50,2014-01-02,2014-01-02,Placebo,Placebo,2014-01-02 00:30:00,NA,2014-01-02,00:30:00,1


In [22]:
colnames(pc_dates)

[1] "STUDYID"  "DOMAIN"   "USUBJID"  "EVID"     "DRUG"     "NFRLT"   
 [7] "PCSEQ"    "PCTESTCD" "PCTEST"   "PCORRES"  "PCORRESU" "PCSTRESC"
[13] "PCSTRESN" "PCSTRESU" "PCNAM"    "PCSPEC"   "PCLLOQ"   "VISIT"   
[19] "VISITNUM" "PCDTC"    "PCDY"     "PCTPT"    "PCTPTNUM" "TRTSDT"  
[25] "TRTSDTM"  "TRT01P"   "TRT01A"   "ADTM"     "ATMF"     "ADT"     
[31] "ATM"      "ADY"

### Get Dosing Information


Here we also create nomimal time from first dose `NFRLT` for `EX` data based on `VISITDY`.


In [23]:
#| label: Dosing

ex_dates <- ex %>%
  derive_vars_merged(
    dataset_add = adsl,
    new_vars = adsl_vars,
    by_vars = exprs(STUDYID, USUBJID)
  ) %>%
  # Keep records with nonzero dose
  filter(EXDOSE > 0) %>%
  # Add time and set missing end date to start date
  # Impute missing time to 00:00:00
  # Note all times are missing for dosing records in this example data
  # Derive Analysis Start and End Dates
  derive_vars_dtm(
    new_vars_prefix = "AST",
    dtc = EXSTDTC,
    time_imputation = "00:00:00"
  ) %>%
  derive_vars_dtm(
    new_vars_prefix = "AEN",
    dtc = EXENDTC,
    time_imputation = "00:00:00"
  ) %>%
  # Derive event ID and nominal relative time from first dose (NFRLT)
  mutate(
    EVID = 1,
    NFRLT = case_when(
      VISITDY == 1 ~ 0,
      TRUE ~ 24 * VISITDY
    )
  ) %>%
  # Set missing end dates to start date
  mutate(AENDTM = case_when(
    is.na(AENDTM) ~ ASTDTM,
    TRUE ~ AENDTM
  )) %>%
  # Derive dates from date/times
  derive_vars_dtm_to_dt(exprs(ASTDTM)) %>%
  derive_vars_dtm_to_dt(exprs(AENDTM))


In [24]:
head(ex_dates,3)

STUDYID,DOMAIN,USUBJID,EXSEQ,EXTRT,EXDOSE,EXDOSU,EXDOSFRM,EXDOSFRQ,EXROUTE,...,TRT01P,TRT01A,ASTDTM,ASTTMF,AENDTM,AENTMF,EVID,NFRLT,ASTDT,AENDT
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<dttm>,<chr>,<dttm>,<chr>,<dbl>,<dbl>,<date>,<date>
CDISCPILOT01,EX,01-701-1028,1,XANOMELINE,54,mg,PATCH,QD,TRANSDERMAL,...,Xanomeline High Dose,Xanomeline High Dose,2013-07-19,H,2013-08-01,H,1,0,2013-07-19,2013-08-01
CDISCPILOT01,EX,01-701-1028,2,XANOMELINE,81,mg,PATCH,QD,TRANSDERMAL,...,Xanomeline High Dose,Xanomeline High Dose,2013-08-02,H,2014-01-06,H,1,336,2013-08-02,2014-01-06
CDISCPILOT01,EX,01-701-1028,3,XANOMELINE,54,mg,PATCH,QD,TRANSDERMAL,...,Xanomeline High Dose,Xanomeline High Dose,2014-01-07,H,2014-01-14,H,1,4032,2014-01-07,2014-01-14


In [25]:
colnames(ex_dates)

[1] "STUDYID"  "DOMAIN"   "USUBJID"  "EXSEQ"    "EXTRT"    "EXDOSE"  
 [7] "EXDOSU"   "EXDOSFRM" "EXDOSFRQ" "EXROUTE"  "VISITNUM" "VISIT"   
[13] "VISITDY"  "EXSTDTC"  "EXENDTC"  "EXSTDY"   "EXENDY"   "TRTSDT"  
[19] "TRTSDTM"  "TRT01P"   "TRT01A"   "ASTDTM"   "ASTTMF"   "AENDTM"  
[25] "AENTMF"   "EVID"     "NFRLT"    "ASTDT"    "AENDT"

### Expand Dosing Records


Since there is a start date and end date for dosing records we need to expand the dosing records between the start date and end date using the function `admiral::create_single_dose_dataset()`.


In [26]:
#| label: Expand

ex_exp <- ex_dates %>%
  create_single_dose_dataset(
    dose_freq = EXDOSFRQ,
    start_date = ASTDT,
    start_datetime = ASTDTM,
    end_date = AENDT,
    end_datetime = AENDTM,
    nominal_time = NFRLT,
    lookup_table = dose_freq_lookup,
    lookup_column = CDISC_VALUE,
    keep_source_vars = exprs(
      STUDYID, USUBJID, EVID, EXDOSFRQ, EXDOSFRM,
      NFRLT, EXDOSE, EXDOSU, EXTRT, ASTDT, ASTDTM, AENDT, AENDTM,
      VISIT, VISITNUM, VISITDY,
      TRT01A, TRT01P, DOMAIN, EXSEQ, !!!adsl_vars
    )
  ) %>%
  # Derive AVISIT based on nominal relative time
  # Derive AVISITN to nominal time in whole days using integer division
  # Define AVISIT based on nominal day
  mutate(
    AVISITN = NFRLT %/% 24 + 1,
    AVISIT = paste("Day", AVISITN),
    ADTM = ASTDTM,
    DRUG = EXTRT
  ) %>%
  # Derive dates and times from datetimes
  derive_vars_dtm_to_dt(exprs(ADTM)) %>%
  derive_vars_dtm_to_tm(exprs(ADTM)) %>%
  derive_vars_dtm_to_tm(exprs(ASTDTM)) %>%
  derive_vars_dtm_to_tm(exprs(AENDTM)) %>%
  derive_vars_dy(reference_date = TRTSDT, source_vars = exprs(ADT))


In [27]:
head(ex_exp,3)

STUDYID,USUBJID,EVID,EXDOSFRQ,EXDOSFRM,NFRLT,EXDOSE,EXDOSU,EXTRT,ASTDT,...,TRTSDTM,AVISITN,AVISIT,ADTM,DRUG,ADT,ATM,ASTTM,AENTM,ADY
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<date>,...,<dttm>,<dbl>,<chr>,<dttm>,<chr>,<date>,<time>,<time>,<time>,<dbl>
CDISCPILOT01,01-701-1028,1,ONCE,PATCH,0,54,mg,XANOMELINE,2013-07-19,...,2013-07-19,1,Day 1,2013-07-19,XANOMELINE,2013-07-19,00:00:00,00:00:00,00:00:00,1
CDISCPILOT01,01-701-1028,1,ONCE,PATCH,24,54,mg,XANOMELINE,2013-07-20,...,2013-07-19,2,Day 2,2013-07-20,XANOMELINE,2013-07-20,00:00:00,00:00:00,00:00:00,2
CDISCPILOT01,01-701-1028,1,ONCE,PATCH,48,54,mg,XANOMELINE,2013-07-21,...,2013-07-19,3,Day 3,2013-07-21,XANOMELINE,2013-07-21,00:00:00,00:00:00,00:00:00,3


In [28]:
colnames(ex_exp)

[1] "STUDYID"  "USUBJID"  "EVID"     "EXDOSFRQ" "EXDOSFRM" "NFRLT"   
 [7] "EXDOSE"   "EXDOSU"   "EXTRT"    "ASTDT"    "ASTDTM"   "AENDT"   
[13] "AENDTM"   "VISIT"    "VISITNUM" "VISITDY"  "TRT01A"   "TRT01P"  
[19] "DOMAIN"   "EXSEQ"    "TRTSDT"   "TRTSDTM"  "AVISITN"  "AVISIT"  
[25] "ADTM"     "DRUG"     "ADT"      "ATM"      "ASTTM"    "AENTM"   
[31] "ADY"

### Find First Dose


In this section we will find the first dose for each subject and drug.


In [29]:
#| label: First Dose

adpc_first_dose <- pc_dates %>%
  derive_vars_merged(
    dataset_add = ex_exp,
    filter_add = (EXDOSE > 0 & !is.na(ADTM)),
    new_vars = exprs(FANLDTM = ADTM),
    order = exprs(ADTM, EXSEQ),
    mode = "first",
    by_vars = exprs(STUDYID, USUBJID, DRUG)
  ) %>%
  filter(!is.na(FANLDTM)) %>%
  # Derive AVISIT based on nominal relative time
  # Derive AVISITN to nominal time in whole days using integer division
  # Define AVISIT based on nominal day
  mutate(
    AVISITN = NFRLT %/% 24 + 1,
    AVISIT = paste("Day", AVISITN),
  )


In [30]:
head(adpc_first_dose, 3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,TRT01P,TRT01A,ADTM,ATMF,ADT,ATM,ADY,FANLDTM,AVISITN,AVISIT
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<dttm>,<chr>,<date>,<time>,<dbl>,<dttm>,<dbl>,<chr>
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,Xanomeline High Dose,Xanomeline High Dose,2013-07-18 23:30:00,NA,2013-07-18,23:30:00,-1,2013-07-19,1,Day 1
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.08,2,XAN,XANOMELINE,0.101566224882241,...,Xanomeline High Dose,Xanomeline High Dose,2013-07-19 00:05:00,NA,2013-07-19,00:05:00,1,2013-07-19,1,Day 1
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.50,3,XAN,XANOMELINE,0.546901773708848,...,Xanomeline High Dose,Xanomeline High Dose,2013-07-19 00:30:00,NA,2013-07-19,00:30:00,1,2013-07-19,1,Day 1


### Find Previous Dose and Next Dose


Use `derive_vars_joined()` to find the previous dose and the next dose.


In [32]:
#| label: Previous Dose and Next Dose

adpc_prev <- adpc_first_dose %>%
  derive_vars_joined(
    dataset_add = ex_exp,
    by_vars = exprs(USUBJID),
    order = exprs(ADTM),
    new_vars = exprs(
      ADTM_prev = ADTM, EXDOSE_prev = EXDOSE, AVISIT_prev = AVISIT,
      AENDTM_prev = AENDTM
    ),
    join_vars = exprs(ADTM),
    join_type = "all",
    filter_add = NULL,
    filter_join = ADTM > ADTM.join,
    mode = "last",
    check_type = "none"
  )

adpc_next <- adpc_prev %>%
  derive_vars_joined(
    dataset_add = ex_exp,
    by_vars = exprs(USUBJID),
    order = exprs(ADTM),
    new_vars = exprs(
      ADTM_next = ADTM, EXDOSE_next = EXDOSE, AVISIT_next = AVISIT,
      AENDTM_next = AENDTM
    ),
    join_vars = exprs(ADTM),
    join_type = "all",
    filter_add = NULL,
    filter_join = ADTM <= ADTM.join,
    mode = "first",
    check_type = "none"
  )


In [33]:
head(adpc_prev,3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,ADT,ATM,ADY,FANLDTM,AVISITN,AVISIT,ADTM_prev,EXDOSE_prev,AVISIT_prev,AENDTM_prev
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<date>,<time>,<dbl>,<dttm>,<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dttm>
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,2013-07-18,23:30:00,-1,2013-07-19,1,Day 1,NA,NA,NA,NA
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.08,2,XAN,XANOMELINE,0.101566224882241,...,2013-07-19,00:05:00,1,2013-07-19,1,Day 1,2013-07-19,54,Day 1,2013-07-19
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.50,3,XAN,XANOMELINE,0.546901773708848,...,2013-07-19,00:30:00,1,2013-07-19,1,Day 1,2013-07-19,54,Day 1,2013-07-19


In [34]:
colnames(adpc_prev)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "EVID"        "DRUG"       
 [6] "NFRLT"       "PCSEQ"       "PCTESTCD"    "PCTEST"      "PCORRES"    
[11] "PCORRESU"    "PCSTRESC"    "PCSTRESN"    "PCSTRESU"    "PCNAM"      
[16] "PCSPEC"      "PCLLOQ"      "VISIT"       "VISITNUM"    "PCDTC"      
[21] "PCDY"        "PCTPT"       "PCTPTNUM"    "TRTSDT"      "TRTSDTM"    
[26] "TRT01P"      "TRT01A"      "ADTM"        "ATMF"        "ADT"        
[31] "ATM"         "ADY"         "FANLDTM"     "AVISITN"     "AVISIT"     
[36] "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev" "AENDTM_prev"

In [35]:
head(adpc_next,3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,AVISITN,AVISIT,ADTM_prev,EXDOSE_prev,AVISIT_prev,AENDTM_prev,ADTM_next,EXDOSE_next,AVISIT_next,AENDTM_next
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dttm>,<dttm>,<dbl>,<chr>,<dttm>
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,1,Day 1,NA,NA,NA,NA,2013-07-19,54,Day 1,2013-07-19
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.08,2,XAN,XANOMELINE,0.101566224882241,...,1,Day 1,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.50,3,XAN,XANOMELINE,0.546901773708848,...,1,Day 1,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20


In [36]:
colnames(adpc_next)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "EVID"        "DRUG"       
 [6] "NFRLT"       "PCSEQ"       "PCTESTCD"    "PCTEST"      "PCORRES"    
[11] "PCORRESU"    "PCSTRESC"    "PCSTRESN"    "PCSTRESU"    "PCNAM"      
[16] "PCSPEC"      "PCLLOQ"      "VISIT"       "VISITNUM"    "PCDTC"      
[21] "PCDY"        "PCTPT"       "PCTPTNUM"    "TRTSDT"      "TRTSDTM"    
[26] "TRT01P"      "TRT01A"      "ADTM"        "ATMF"        "ADT"        
[31] "ATM"         "ADY"         "FANLDTM"     "AVISITN"     "AVISIT"     
[36] "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev" "AENDTM_prev" "ADTM_next"  
[41] "EXDOSE_next" "AVISIT_next" "AENDTM_next"

### Find Previous and Next Nominal Dose


Use the same method to find the previous and next nominal times.


In [37]:
#| label: Previous Nominal Dose

adpc_nom_prev <- adpc_next %>%
  derive_vars_joined(
    dataset_add = ex_exp,
    by_vars = exprs(USUBJID),
    order = exprs(NFRLT),
    new_vars = exprs(NFRLT_prev = NFRLT),
    join_vars = exprs(NFRLT),
    join_type = "all",
    filter_add = NULL,
    filter_join = NFRLT > NFRLT.join,
    mode = "last",
    check_type = "none"
  )

adpc_nom_next <- adpc_nom_prev %>%
  derive_vars_joined(
    dataset_add = ex_exp,
    by_vars = exprs(USUBJID),
    order = exprs(NFRLT),
    new_vars = exprs(NFRLT_next = NFRLT),
    join_vars = exprs(NFRLT),
    join_type = "all",
    filter_add = NULL,
    filter_join = NFRLT <= NFRLT.join,
    mode = "first",
    check_type = "none"
  )


In [38]:
head(adpc_nom_prev,3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,AVISIT,ADTM_prev,EXDOSE_prev,AVISIT_prev,AENDTM_prev,ADTM_next,EXDOSE_next,AVISIT_next,AENDTM_next,NFRLT_prev
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<chr>,<dttm>,<dbl>,<chr>,<dttm>,<dttm>,<dbl>,<chr>,<dttm>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,Day 1,NA,NA,NA,NA,2013-07-19,54,Day 1,2013-07-19,NA
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.08,2,XAN,XANOMELINE,0.101566224882241,...,Day 1,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20,0
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.50,3,XAN,XANOMELINE,0.546901773708848,...,Day 1,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20,0


In [39]:
head(adpc_nom_next,3)

STUDYID,DOMAIN,USUBJID,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,PCTEST,PCORRES,...,ADTM_prev,EXDOSE_prev,AVISIT_prev,AENDTM_prev,ADTM_next,EXDOSE_next,AVISIT_next,AENDTM_next,NFRLT_prev,NFRLT_next
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<dttm>,<dbl>,<chr>,<dttm>,<dttm>,<dbl>,<chr>,<dttm>,<dbl>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.00,1,XAN,XANOMELINE,<BLQ,...,NA,NA,NA,NA,2013-07-19,54,Day 1,2013-07-19,NA,0
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.08,2,XAN,XANOMELINE,0.101566224882241,...,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20,0,24
CDISCPILOT01,PC,01-701-1028,0,XANOMELINE,0.50,3,XAN,XANOMELINE,0.546901773708848,...,2013-07-19,54,Day 1,2013-07-19,2013-07-20,54,Day 2,2013-07-20,0,24


### Combine PC and EX Data


Combine `PC` and `EX` records and derive the additional relative time variables.


In [41]:
#| label: Combine

adpc_arrlt <- bind_rows(adpc_nom_next, ex_exp) %>%
  group_by(USUBJID, DRUG) %>%
  mutate(
    FANLDTM = min(FANLDTM, na.rm = TRUE),
    min_NFRLT = min(NFRLT_prev, na.rm = TRUE),
    maxdate = max(ADT[EVID == 0], na.rm = TRUE), .after = USUBJID
  ) %>%
  arrange(USUBJID, ADTM) %>%
  ungroup() %>%
  filter(ADT <= maxdate) %>%
  # Derive Actual Relative Time from First Dose (AFRLT)
  derive_vars_duration(
    new_var = AFRLT,
    start_date = FANLDTM,
    end_date = ADTM,
    out_unit = "hours",
    floor_in = FALSE,
    add_one = FALSE
  ) %>%
  # Derive Actual Relative Time from Reference Dose (ARRLT)
  derive_vars_duration(
    new_var = ARRLT,
    start_date = ADTM_prev,
    end_date = ADTM,
    out_unit = "hours",
    floor_in = FALSE,
    add_one = FALSE
  ) %>%
  # Derive Actual Relative Time from Next Dose (AXRLT not kept)
  derive_vars_duration(
    new_var = AXRLT,
    start_date = ADTM_next,
    end_date = ADTM,
    out_unit = "hours",
    floor_in = FALSE,
    add_one = FALSE
  ) %>%
  mutate(
    ARRLT = case_when(
      EVID == 1 ~ 0,
      is.na(ARRLT) ~ AXRLT,
      TRUE ~ ARRLT
    ),
    # Derive Reference Dose Date
    PCRFTDTM = case_when(
      EVID == 1 ~ ADTM,
      is.na(ADTM_prev) ~ ADTM_next,
      TRUE ~ ADTM_prev
    )
  ) %>%
  # Derive dates and times from datetimes
  derive_vars_dtm_to_dt(exprs(FANLDTM)) %>%
  derive_vars_dtm_to_tm(exprs(FANLDTM)) %>%
  derive_vars_dtm_to_dt(exprs(PCRFTDTM)) %>%
  derive_vars_dtm_to_tm(exprs(PCRFTDTM))


In [42]:
head(adpc_arrlt, 3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,ASTTM,AENTM,AFRLT,ARRLT,AXRLT,PCRFTDTM,FANLDT,FANLTM,PCRFTDT,PCRFTTM
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<time>,<time>,<dbl>,<dbl>,<dbl>,<dttm>,<date>,<time>,<date>,<time>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,NA,NA,-0.50000000,-0.50000000,-0.50000,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,00:00:00,00:00:00,0.00000000,0.00000000,NA,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,NA,NA,0.08333333,0.08333333,-23.91667,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00


In [43]:
colnames(adpc_arrlt)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "min_NFRLT"   "maxdate"    
 [6] "EVID"        "DRUG"        "NFRLT"       "PCSEQ"       "PCTESTCD"   
[11] "PCTEST"      "PCORRES"     "PCORRESU"    "PCSTRESC"    "PCSTRESN"   
[16] "PCSTRESU"    "PCNAM"       "PCSPEC"      "PCLLOQ"      "VISIT"      
[21] "VISITNUM"    "PCDTC"       "PCDY"        "PCTPT"       "PCTPTNUM"   
[26] "TRTSDT"      "TRTSDTM"     "TRT01P"      "TRT01A"      "ADTM"       
[31] "ATMF"        "ADT"         "ATM"         "ADY"         "FANLDTM"    
[36] "AVISITN"     "AVISIT"      "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev"
[41] "AENDTM_prev" "ADTM_next"   "EXDOSE_next" "AVISIT_next" "AENDTM_next"
[46] "NFRLT_prev"  "NFRLT_next"  "EXDOSFRQ"    "EXDOSFRM"    "EXDOSE"     
[51] "EXDOSU"      "EXTRT"       "ASTDT"       "ASTDTM"      "AENDT"      
[56] "AENDTM"      "VISITDY"     "EXSEQ"       "ASTTM"       "AENTM"      
[61] "AFRLT"       "ARRLT"       "AXRLT"       "PCRFTDTM"    "FANLDT"     
[66] "FANLTM"      "PCRFTDT"     "PCRFTTM"

### Derive Nominal Reference


For nominal relative times we calculate the nominal relative time to reference dose `NRRLT`.


In [45]:
#| label: Nominal Reference

# Derive Nominal Relative Time from Reference Dose (NRRLT)

adpc_nrrlt <- adpc_arrlt %>%
  mutate(
    NRRLT = case_when(
      EVID == 1 ~ 0,
      is.na(NFRLT_prev) ~ NFRLT - min_NFRLT,
      TRUE ~ NFRLT - NFRLT_prev
    ),
    NXRLT = case_when(
      EVID == 1 ~ 0,
      TRUE ~ NFRLT - NFRLT_next
    )
  )


In [46]:
head(adpc_nrrlt, 3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,AFRLT,ARRLT,AXRLT,PCRFTDTM,FANLDT,FANLTM,PCRFTDT,PCRFTTM,NRRLT,NXRLT
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<dbl>,<dbl>,<dbl>,<dttm>,<date>,<time>,<date>,<time>,<dbl>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,-0.50000000,-0.50000000,-0.50000,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00,0.00,0.00
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,0.00000000,0.00000000,NA,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00,0.00,0.00
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,0.08333333,0.08333333,-23.91667,2013-07-19,2013-07-19,00:00:00,2013-07-19,00:00:00,0.08,-23.92


In [47]:
colnames(adpc_nrrlt)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "min_NFRLT"   "maxdate"    
 [6] "EVID"        "DRUG"        "NFRLT"       "PCSEQ"       "PCTESTCD"   
[11] "PCTEST"      "PCORRES"     "PCORRESU"    "PCSTRESC"    "PCSTRESN"   
[16] "PCSTRESU"    "PCNAM"       "PCSPEC"      "PCLLOQ"      "VISIT"      
[21] "VISITNUM"    "PCDTC"       "PCDY"        "PCTPT"       "PCTPTNUM"   
[26] "TRTSDT"      "TRTSDTM"     "TRT01P"      "TRT01A"      "ADTM"       
[31] "ATMF"        "ADT"         "ATM"         "ADY"         "FANLDTM"    
[36] "AVISITN"     "AVISIT"      "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev"
[41] "AENDTM_prev" "ADTM_next"   "EXDOSE_next" "AVISIT_next" "AENDTM_next"
[46] "NFRLT_prev"  "NFRLT_next"  "EXDOSFRQ"    "EXDOSFRM"    "EXDOSE"     
[51] "EXDOSU"      "EXTRT"       "ASTDT"       "ASTDTM"      "AENDT"      
[56] "AENDTM"      "VISITDY"     "EXSEQ"       "ASTTM"       "AENTM"      
[61] "AFRLT"       "ARRLT"       "AXRLT"       "PCRFTDTM"    "FANLDT"     
[66] "FANLTM"      "PCRFTDT"     "PCRFTTM"     "NRRLT"       "NXRLT"

### Derive Analysis Variables


Here we derive the analysis variables such as `AVAL` and `ATPTREF`.


In [49]:
#| label: Analysis Variables

adpc_aval <- adpc_nrrlt %>%
  mutate(
    PARCAT1 = PCSPEC,
    ATPTN = case_when(
      EVID == 1 ~ 0,
      TRUE ~ PCTPTNUM
    ),
    ATPT = case_when(
      EVID == 1 ~ "Dose",
      TRUE ~ PCTPT
    ),
    ATPTREF = case_when(
      EVID == 1 ~ AVISIT,
      is.na(AVISIT_prev) ~ AVISIT_next,
      TRUE ~ AVISIT_prev
    ),
    # Derive baseline flag for pre-dose records
    ABLFL = case_when(
      ATPT == "Pre-dose" ~ "Y",
      TRUE ~ NA_character_
    ),
    # Derive BASETYPE
    BASETYPE = paste(ATPTREF, "Baseline"),

    # Derive Actual Dose
    DOSEA = case_when(
      EVID == 1 ~ EXDOSE,
      is.na(EXDOSE_prev) ~ EXDOSE_next,
      TRUE ~ EXDOSE_prev
    ),
    # Derive Planned Dose
    DOSEP = case_when(
      TRT01P == "Xanomeline High Dose" ~ 81,
      TRT01P == "Xanomeline Low Dose" ~ 54
    ),
    DOSEU = "mg",
  ) %>%
  # Derive relative time units
  mutate(
    FRLTU = "h",
    RRLTU = "h",
    # Derive PARAMCD
    PARAMCD = coalesce(PCTESTCD, "DOSE"),
    ALLOQ = PCLLOQ,
    # Derive AVAL
    AVAL = case_when(
      EVID == 1 ~ EXDOSE,
      PCSTRESC == "<BLQ" & NFRLT == 0 ~ 0,
      PCSTRESC == "<BLQ" & NFRLT > 0 ~ 0.5 * ALLOQ,
      TRUE ~ PCSTRESN
    ),
    AVALU = case_when(
      EVID == 1 ~ EXDOSU,
      TRUE ~ PCSTRESU
    ),
    AVALCAT1 = if_else(PCSTRESC == "<BLQ", PCSTRESC, prettyNum(signif(AVAL, digits = 3))),
  ) %>%
  # Add SRCSEQ
  mutate(
    SRCDOM = DOMAIN,
    SRCVAR = "SEQ",
    SRCSEQ = coalesce(PCSEQ, EXSEQ)
  )


In [50]:
head(adpc_aval, 3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,FRLTU,RRLTU,PARAMCD,ALLOQ,AVAL,AVALU,AVALCAT1,SRCDOM,SRCVAR,SRCSEQ
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,h,h,XAN,0.01,0.0000000,ug/ml,<BLQ,PC,SEQ,1
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,h,h,DOSE,NA,54.0000000,mg,NA,EX,SEQ,1
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,h,h,XAN,0.01,0.1015662,ug/ml,0.102,PC,SEQ,2


### Derive DTYPE Copy Records


The CDISC ADaM Implementation Guide for Non-compartmental Analysis uses duplicated records for analysis when a record needs to be used in more than one way. In this example the 24 hour post-dose record will also be used a the pre-dose record for the "Day 2" dose.


In [52]:
#| label: DTYPE

dtype <- adpc_aval %>%
  filter(NFRLT > 0 & NXRLT == 0 & EVID == 0 & !is.na(AVISIT_next)) %>%
  select(-PCRFTDT, -PCRFTTM) %>%
  # Re-derive variables in for DTYPE copy records
  mutate(
    ABLFL = NA_character_,
    ATPTREF = AVISIT_next,
    ARRLT = AXRLT,
    NRRLT = NXRLT,
    PCRFTDTM = ADTM_next,
    DOSEA = EXDOSE_next,
    BASETYPE = paste(AVISIT_next, "Baseline"),
    ATPT = "Pre-dose",
    ATPTN = NFRLT,
    ABLFL = "Y",
    DTYPE = "COPY"
  ) %>%
  derive_vars_dtm_to_dt(exprs(PCRFTDTM)) %>%
  derive_vars_dtm_to_tm(exprs(PCRFTDTM))


In [53]:
head(dtype,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,ALLOQ,AVAL,AVALU,AVALCAT1,SRCDOM,SRCVAR,SRCSEQ,DTYPE,PCRFTDT,PCRFTTM
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<date>,<time>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,24,12,XAN,...,0.01,0.01070627,ug/ml,0.0107,PC,SEQ,12,COPY,2013-07-20,00:00:00
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,48,14,XAN,...,0.01,0.00500000,ug/ml,<BLQ,PC,SEQ,14,COPY,2013-07-21,00:00:00
CDISCPILOT01,PC,01-701-1033,0,2014-03-20,0,XANOMELINE,24,12,XAN,...,0.01,0.01783681,ug/ml,0.0178,PC,SEQ,12,COPY,2014-03-19,00:00:00


### Combine Original and DTYPE Copy


Now the duplicated records are combined with the original records.


In [55]:
#| label: Combine DTYPE

adpc_dtype <- bind_rows(adpc_aval, dtype) %>%
  arrange(STUDYID, USUBJID, BASETYPE, ADTM, NFRLT) %>%
  mutate(
    # Derive MRRLT, ANL01FL and ANL02FL
    MRRLT = if_else(ARRLT < 0, 0, ARRLT),
    ANL01FL = "Y",
    ANL02FL = if_else(is.na(DTYPE), "Y", NA_character_),
  )


In [56]:
head(adpc_dtype,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,AVAL,AVALU,AVALCAT1,SRCDOM,SRCVAR,SRCSEQ,DTYPE,MRRLT,ANL01FL,ANL02FL
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,0.0000000,ug/ml,<BLQ,PC,SEQ,1,NA,0.00000000,Y,Y
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,54.0000000,mg,NA,EX,SEQ,1,NA,0.00000000,Y,Y
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,0.1015662,ug/ml,0.102,PC,SEQ,2,NA,0.08333333,Y,Y


In [57]:
colnames(adpc_dtype)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "min_NFRLT"   "maxdate"    
 [6] "EVID"        "DRUG"        "NFRLT"       "PCSEQ"       "PCTESTCD"   
[11] "PCTEST"      "PCORRES"     "PCORRESU"    "PCSTRESC"    "PCSTRESN"   
[16] "PCSTRESU"    "PCNAM"       "PCSPEC"      "PCLLOQ"      "VISIT"      
[21] "VISITNUM"    "PCDTC"       "PCDY"        "PCTPT"       "PCTPTNUM"   
[26] "TRTSDT"      "TRTSDTM"     "TRT01P"      "TRT01A"      "ADTM"       
[31] "ATMF"        "ADT"         "ATM"         "ADY"         "FANLDTM"    
[36] "AVISITN"     "AVISIT"      "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev"
[41] "AENDTM_prev" "ADTM_next"   "EXDOSE_next" "AVISIT_next" "AENDTM_next"
[46] "NFRLT_prev"  "NFRLT_next"  "EXDOSFRQ"    "EXDOSFRM"    "EXDOSE"     
[51] "EXDOSU"      "EXTRT"       "ASTDT"       "ASTDTM"      "AENDT"      
[56] "AENDTM"      "VISITDY"     "EXSEQ"       "ASTTM"       "AENTM"      
[61] "AFRLT"       "ARRLT"       "AXRLT"       "PCRFTDTM"    "FANLDT"     
[66] "FANLTM"      "PCRFTDT"     "PCRFTTM"     "NRRLT"       "NXRLT"      
[71] "PARCAT1"     "ATPTN"       "ATPT"        "ATPTREF"     "ABLFL"      
[76] "BASETYPE"    "DOSEA"       "DOSEP"       "DOSEU"       "FRLTU"      
[81] "RRLTU"       "PARAMCD"     "ALLOQ"       "AVAL"        "AVALU"      
[86] "AVALCAT1"    "SRCDOM"      "SRCVAR"      "SRCSEQ"      "DTYPE"      
[91] "MRRLT"       "ANL01FL"     "ANL02FL"

### Derive BASE and CHG


In [58]:
#| label: BASE

# ---- Derive BASE and Calculate Change from Baseline ----

adpc_base <- adpc_dtype %>%
  derive_var_base(
    by_vars = exprs(STUDYID, USUBJID, PARAMCD, PARCAT1, BASETYPE),
    source_var = AVAL,
    new_var = BASE,
    filter = ABLFL == "Y"
  )

adpc_chg <- derive_var_chg(adpc_base)


In [59]:
head(adpc_base,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,AVALU,AVALCAT1,SRCDOM,SRCVAR,SRCSEQ,DTYPE,MRRLT,ANL01FL,ANL02FL,BASE
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,ug/ml,<BLQ,PC,SEQ,1,NA,0.00000000,Y,Y,0
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,mg,NA,EX,SEQ,1,NA,0.00000000,Y,Y,NA
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,ug/ml,0.102,PC,SEQ,2,NA,0.08333333,Y,Y,0


In [60]:
colnames(adpc_base)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "min_NFRLT"   "maxdate"    
 [6] "EVID"        "DRUG"        "NFRLT"       "PCSEQ"       "PCTESTCD"   
[11] "PCTEST"      "PCORRES"     "PCORRESU"    "PCSTRESC"    "PCSTRESN"   
[16] "PCSTRESU"    "PCNAM"       "PCSPEC"      "PCLLOQ"      "VISIT"      
[21] "VISITNUM"    "PCDTC"       "PCDY"        "PCTPT"       "PCTPTNUM"   
[26] "TRTSDT"      "TRTSDTM"     "TRT01P"      "TRT01A"      "ADTM"       
[31] "ATMF"        "ADT"         "ATM"         "ADY"         "FANLDTM"    
[36] "AVISITN"     "AVISIT"      "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev"
[41] "AENDTM_prev" "ADTM_next"   "EXDOSE_next" "AVISIT_next" "AENDTM_next"
[46] "NFRLT_prev"  "NFRLT_next"  "EXDOSFRQ"    "EXDOSFRM"    "EXDOSE"     
[51] "EXDOSU"      "EXTRT"       "ASTDT"       "ASTDTM"      "AENDT"      
[56] "AENDTM"      "VISITDY"     "EXSEQ"       "ASTTM"       "AENTM"      
[61] "AFRLT"       "ARRLT"       "AXRLT"       "PCRFTDTM"    "FANLDT"     
[66] "FANLTM"      "PCRFTDT"     "PCRFTTM"     "NRRLT"       "NXRLT"      
[71] "PARCAT1"     "ATPTN"       "ATPT"        "ATPTREF"     "ABLFL"      
[76] "BASETYPE"    "DOSEA"       "DOSEP"       "DOSEU"       "FRLTU"      
[81] "RRLTU"       "PARAMCD"     "ALLOQ"       "AVAL"        "AVALU"      
[86] "AVALCAT1"    "SRCDOM"      "SRCVAR"      "SRCSEQ"      "DTYPE"      
[91] "MRRLT"       "ANL01FL"     "ANL02FL"     "BASE"

### Derive `PARAM` with `{metatools}`


Here we derive `PARAM` and `PARAMN` using `create_var_from_codelist()` from `{metatools}`.


In [61]:
#| label: ASEQ

# ---- Add ASEQ ----

adpc_aseq <- adpc_chg %>%
  # Calculate ASEQ
  derive_var_obs_number(
    new_var = ASEQ,
    by_vars = exprs(STUDYID, USUBJID),
    order = exprs(ADTM, BASETYPE, EVID, AVISITN, ATPTN, PARCAT1, DTYPE),
    check_type = "error"
  ) %>%
  # Derive PARAM and PARAMN using metatools
  create_var_from_codelist(metacore, input_var = PARAMCD, out_var = PARAM) %>%
  create_var_from_codelist(metacore, input_var = PARAMCD, out_var = PARAMN)


In [62]:
head(adpc_aseq,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,SRCSEQ,DTYPE,MRRLT,ANL01FL,ANL02FL,BASE,CHG,ASEQ,PARAM,PARAMN
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,1,NA,0.00000000,Y,Y,0,0.0000000,1,Pharmacokinetic concentration of Xanomeline,1
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,1,NA,0.00000000,Y,Y,NA,NA,2,Xanomeline Patch Dose,2
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,2,NA,0.08333333,Y,Y,0,0.1015662,3,Pharmacokinetic concentration of Xanomeline,1


### Derive Additional Baselines


Here we derive additional baseline values from `VS` for baseline height `HTBL` and weight `WTBL` and compute the body mass index (BMI) with `compute_bmi()`.


In [63]:
#| label: Baselines
#---- Derive additional baselines from VS ----

adpc_baselines <- adpc_aseq %>%
  derive_vars_merged(
    dataset_add = vs,
    filter_add = VSTESTCD == "HEIGHT",
    by_vars = exprs(STUDYID, USUBJID),
    new_vars = exprs(HTBL = VSSTRESN, HTBLU = VSSTRESU)
  ) %>%
  derive_vars_merged(
    dataset_add = vs,
    filter_add = VSTESTCD == "WEIGHT" & VSBLFL == "Y",
    by_vars = exprs(STUDYID, USUBJID),
    new_vars = exprs(WTBL = VSSTRESN, WTBLU = VSSTRESU)
  ) %>%
  mutate(
    BMIBL = compute_bmi(height = HTBL, weight = WTBL),
    BMIBLU = "kg/m^2"
  )


In [64]:
head(adpc_baselines,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,CHG,ASEQ,PARAM,PARAMN,HTBL,HTBLU,WTBL,WTBLU,BMIBL,BMIBLU
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<dbl>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,0.0000000,1,Pharmacokinetic concentration of Xanomeline,1,177.8,cm,99.34,kg,31.42394,kg/m^2
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,NA,2,Xanomeline Patch Dose,2,177.8,cm,99.34,kg,31.42394,kg/m^2
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,0.1015662,3,Pharmacokinetic concentration of Xanomeline,1,177.8,cm,99.34,kg,31.42394,kg/m^2


### Combine with ADSL


If needed, the other `ADSL` variables can now be added:


In [65]:
#| label: Combine with ADSL
# ---- Add all ADSL variables ----

# Add all ADSL variables
adpc_prefinal <- adpc_baselines %>%
  derive_vars_merged(
    dataset_add = select(adsl, !!!negate_vars(adsl_vars)),
    by_vars = exprs(STUDYID, USUBJID)
  )


In [66]:
head(adpc_prefinal,3)

STUDYID,DOMAIN,USUBJID,min_NFRLT,maxdate,EVID,DRUG,NFRLT,PCSEQ,PCTESTCD,...,DTHCGR1,LSTALVDT,SAFFL,RACEGR1,AGEGR1,REGION1,LDDTHGR1,DTH30FL,DTHA30FL,DTHB30FL
<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<chr>,<dbl>,<int>,<chr>,...,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.00,1,XAN,...,NA,2014-01-14,Y,White,>64,NA,NA,NA,NA,NA
CDISCPILOT01,EX,01-701-1028,0,2013-07-21,1,XANOMELINE,0.00,NA,NA,...,NA,2014-01-14,Y,White,>64,NA,NA,NA,NA,NA
CDISCPILOT01,PC,01-701-1028,0,2013-07-21,0,XANOMELINE,0.08,2,XAN,...,NA,2014-01-14,Y,White,>64,NA,NA,NA,NA,NA


In [67]:
colnames(adpc_prefinal)

[1] "STUDYID"     "DOMAIN"      "USUBJID"     "min_NFRLT"   "maxdate"    
  [6] "EVID"        "DRUG"        "NFRLT"       "PCSEQ"       "PCTESTCD"   
 [11] "PCTEST"      "PCORRES"     "PCORRESU"    "PCSTRESC"    "PCSTRESN"   
 [16] "PCSTRESU"    "PCNAM"       "PCSPEC"      "PCLLOQ"      "VISIT"      
 [21] "VISITNUM"    "PCDTC"       "PCDY"        "PCTPT"       "PCTPTNUM"   
 [26] "TRTSDT"      "TRTSDTM"     "TRT01P"      "TRT01A"      "ADTM"       
 [31] "ATMF"        "ADT"         "ATM"         "ADY"         "FANLDTM"    
 [36] "AVISITN"     "AVISIT"      "ADTM_prev"   "EXDOSE_prev" "AVISIT_prev"
 [41] "AENDTM_prev" "ADTM_next"   "EXDOSE_next" "AVISIT_next" "AENDTM_next"
 [46] "NFRLT_prev"  "NFRLT_next"  "EXDOSFRQ"    "EXDOSFRM"    "EXDOSE"     
 [51] "EXDOSU"      "EXTRT"       "ASTDT"       "ASTDTM"      "AENDT"      
 [56] "AENDTM"      "VISITDY"     "EXSEQ"       "ASTTM"       "AENTM"      
 [61] "AFRLT"       "ARRLT"       "AXRLT"       "PCRFTDTM"    "FANLDT"     
 [66] "FANLTM"      "PCRFTDT"     "PCRFTTM"     "NRRLT"       "NXRLT"      
 [71] "PARCAT1"     "ATPTN"       "ATPT"        "ATPTREF"     "ABLFL"      
 [76] "BASETYPE"    "DOSEA"       "DOSEP"       "DOSEU"       "FRLTU"      
 [81] "RRLTU"       "PARAMCD"     "ALLOQ"       "AVAL"        "AVALU"      
 [86] "AVALCAT1"    "SRCDOM"      "SRCVAR"      "SRCSEQ"      "DTYPE"      
 [91] "MRRLT"       "ANL01FL"     "ANL02FL"     "BASE"        "CHG"        
 [96] "ASEQ"        "PARAM"       "PARAMN"      "HTBL"        "HTBLU"      
[101] "WTBL"        "WTBLU"       "BMIBL"       "BMIBLU"      "SUBJID"     
[106] "RFSTDTC"     "RFENDTC"     "RFXSTDTC"    "RFXENDTC"    "RFICDTC"    
[111] "RFPENDTC"    "DTHDTC"      "DTHFL"       "SITEID"      "AGE"        
[116] "AGEU"        "SEX"         "RACE"        "ETHNIC"      "ARMCD"      
[121] "ARM"         "ACTARMCD"    "ACTARM"      "COUNTRY"     "DMDTC"      
[126] "DMDY"        "TRTSTMF"     "TRTEDTM"     "TRTETMF"     "TRTEDT"     
[131] "TRTDURD"     "SCRFDT"      "EOSDT"       "EOSSTT"      "FRVDT"      
[136] "RANDDT"      "DTHDT"       "DTHDTF"      "DTHADY"      "LDDTHELD"   
[141] "DTHCAUS"     "DTHDOM"      "DTHCGR1"     "LSTALVDT"    "SAFFL"      
[146] "RACEGR1"     "AGEGR1"      "REGION1"     "LDDTHGR1"    "DTH30FL"    
[151] "DTHA30FL"    "DTHB30FL"

## Check Data With metacore and metatools


We use `{metacore}` objects with `{metatools}` functions to perform a number of checks on the data. We will drop variables not in the specs and make sure all the variables from the specs are included.


In [68]:
#| label: Metacore
#| warning: false
# Apply metadata and perform associated checks ----
adpc <- adpc_prefinal %>%
  drop_unspec_vars(metacore) %>% # Drop unspecified variables from specs
  check_variables(metacore) %>% # Check all variables specified are present and no more
  check_ct_data(metacore) %>% # Checks all variables with CT only contain values within the CT
  order_cols(metacore) %>% # Orders the columns according to the spec
  sort_by_key(metacore) # Sorts the rows by the sort keys


The following variable(s) were dropped:
  DOMAIN
  min_NFRLT
  maxdate
  EVID
  DRUG
  PCSEQ
  ADTM_prev
  EXDOSE_prev
  AVISIT_prev
  AENDTM_prev
  ADTM_next
  EXDOSE_next
  AVISIT_next
  AENDTM_next
  NFRLT_prev
  NFRLT_next
  EXDOSFRQ
  EXDOSFRM
  EXDOSE
  EXDOSU
  EXTRT
  VISITDY
  EXSEQ
  AXRLT
  NXRLT
  PARCAT1
  DTHDTF
  DTHCAUS
  DTHDOM
  DTHCGR1

No missing or extra variables



## Apply Labels and Formats with xportr


Using `{xportr}` we check variable type, assign variable lenght, add variable labels, add variable formats, and save a transport file. At the end you could add a call to `xportr::xportr_write()` to produce the XPT file.


In [69]:
#| label: xportr
#| warning: false
dir <- tempdir() # Change to whichever directory you want to save the dataset in

adpc_xpt <- adpc %>%
  xportr_type(metacore, domain = "ADPC") %>% # Coerce variable type to match spec
  xportr_length(metacore) %>% # Assigns SAS length from a variable level metadata
  xportr_label(metacore) %>% # Assigns variable label from metacore specifications
  xportr_format(metacore) %>% # Assigns variable format from metacore specifications
  xportr_df_label(metacore) %>% # Assigns dataset label from metacore specifications
  xportr_write(file.path(dir, "adpc.xpt")) # Write xpt v5 transport file




-- Variable type mismatches found. --



v 5 variables coerced



In [70]:
head(adpc_xpt,3)

,STUDYID,SITEID,USUBJID,SUBJID,COUNTRY,AGE,AGEU,AGEGR1,SEX,RACE,...,WTBL,HTBL,BMIBL,WTBLU,HTBLU,BMIBLU,ASEQ,SRCDOM,SRCVAR,SRCSEQ
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
1,CDISCPILOT01,701,01-701-1028,1028,USA,71,YEARS,>64,M,WHITE,...,99.34,177.8,31.42394,kg,cm,kg/m^2,2,EX,SEQ,1
2,CDISCPILOT01,701,01-701-1028,1028,USA,71,YEARS,>64,M,WHITE,...,99.34,177.8,31.42394,kg,cm,kg/m^2,18,EX,SEQ,1
3,CDISCPILOT01,701,01-701-1028,1028,USA,71,YEARS,>64,M,WHITE,...,99.34,177.8,31.42394,kg,cm,kg/m^2,23,EX,SEQ,1
